<a href="https://colab.research.google.com/github/Srikarmk/Aurora-GP/blob/main/AURORA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gpytorch
import torch
import numpy as np
import gpytorch
import time
from sklearn.metrics import mean_squared_error

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.9/279.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.9 MB/s eta 0:00:00


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def arr_to_tensor(arr):
  return torch.tensor(arr, dtype = torch.float32).to(device)

In [5]:
class exactGP(gpytorch.models.ExactGP):
  def __init__(self, train_x, train_y, likelihood):
    super(exactGP, self).__init__(train_x, train_y, likelihood)
    self.mean_module = gpytorch.means.ConstantMean()
    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

  def forward(self, x):
    mean_x = self.mean_module(x)
    cov_x = self.covar_module(x)
    return gpytorch.distributions.MultivariateNormal(mean_x, cov_x)

In [6]:
from gpytorch.likelihoods import likelihood
def train_exactGP(train_x, train_y, epochs=10):
  likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
  model = exactGP(train_x, train_y, likelihood).to(device)

  model.train()
  likelihood.train()

  optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
  start_time = time.time()

  for i in range(epochs):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    optimizer.step()

    if (i+1) % 10 == 0:
      print(f"Epoch: {i+1}/{epochs}, Loss: {loss.item()}")

  training_time = time.time() - start_time
  print(f"Training time - {training_time:.2f}s")

  return model, likelihood, training_time

In [7]:
def gp_eval(model, likelihood, test_x, test_y):
  model.eval()
  likelihood.eval()

  with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))
    mean_pred = observed_pred.mean
    var_pred = observed_pred.variance
    std_pred = var_pred.sqrt()
    lower = mean_pred - 2*std_pred
    upper = mean_pred + 2*std_pred

    rmse = torch.sqrt(torch.mean((mean_pred - test_y)**2)).item()
    nll = -observed_pred.log_prob(test_y).mean().item()
    calibration = ((test_y >= lower) & (test_y <= upper)).float().mean().item()


    return rmse, nll, calibration

In [14]:
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import train_test_split
#def load_data(filepath, subset_size=None, test_size=0.2):
 # data = np.load(filepath)
  #X = data['X']
  #y = data['y']
  #ft_names = data['feature_names']

  #print(f"Features: {ft_names}")
  #print(f"Shape: X={X.shape}, y={y.shape}")

  #if subset_size is not None and subset_size < len(X):
   # indices = np.random.RandomState(42).choice(len(X), size=subset_size, replace=False)
    #X = X[indices]
    #y = y[indices]
    #print(f"Subset: X={X.shape}, y={y.shape}")

  #x_scale = StandardScaler()
  #y_scale = StandardScaler()
  #X_scaled = x_scale.fit_transform(X)
  #y_scaled = y_scale.fit_transform(y.reshape(-1, 1)).flatten()

  #X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X_scaled, y_scaled, test_size=test_size, random_state=42)

  #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  #X_train = torch.tensor(X_train_np, dtype=torch.float32).to(device)
  #X_test = torch.tensor(X_test_np, dtype=torch.float32).to(device)
  #y_train = torch.tensor(y_train_np, dtype=torch.float32).to(device)
  #y_test = torch.tensor(y_test_np, dtype=torch.float32).to(device)

  #return X_train, X_test, y_train, y_test, x_scale, y_scale

In [36]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
def load_data(filepath, subset_size=None, test_size=0.2, random_state=42):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  data = np.load(filepath)
  X = data['X']
  y = data['y']
  ft_names = data['feature_names']
  print(f"Shape: X={X.shape}, y={y.shape}")

  if subset_size is not None and subset_size < len(X):
    indices = np.random.RandomState(random_state).choice(len(X), size=subset_size, replace=False)
    X = X[indices]
    y = y[indices]
    print(f"Subset: X={X.shape}, y={y.shape}")

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

  x_scale = StandardScaler()
  y_scale = StandardScaler()
  X_train = x_scale.fit_transform(X_train)
  X_test = x_scale.transform(X_test)
  y_train = y_scale.fit_transform(y_train.reshape(-1, 1)).flatten()
  y_test = y_scale.transform(y_test.reshape(-1, 1)).flatten()

  train_x = torch.tensor(X_train, dtype=torch.float32).to(device)
  test_x = torch.tensor(X_test, dtype=torch.float32).to(device)
  train_y = torch.tensor(y_train, dtype=torch.float32).to(device)
  test_y = torch.tensor(y_test, dtype=torch.float32).to(device)

  return train_x, test_x, train_y, test_y, x_scale, y_scale

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
print("ROBOT ARM")
load_data(filepath="/content/drive/MyDrive/datasets/robot_arm.npz", subset_size=None, test_size=0.2)

ROBOT ARM
Shape: X=(8192, 8), y=(8192,)


(tensor([[ 0.8444, -0.5209,  0.8397,  ..., -1.5918,  0.2164, -1.5939],
         [ 1.3718, -1.5744,  0.8551,  ...,  0.8812, -0.9470, -0.8579],
         [ 1.6893,  0.0151, -1.6489,  ..., -1.0079,  0.7043, -1.0411],
         ...,
         [-0.9737, -0.1102,  0.0978,  ...,  0.1581, -0.2764, -0.5299],
         [ 1.0023, -0.0682,  1.3690,  ..., -1.3995, -1.0722, -0.9902],
         [-0.0949,  1.0117, -0.8734,  ..., -1.1463,  0.8837,  0.3538]],
        device='cuda:0'),
 tensor([[ 0.2230, -0.4538, -1.4178,  ...,  0.1198, -0.5327, -0.5034],
         [-1.5587,  1.0166, -0.8154,  ...,  0.4192, -0.8474,  1.0298],
         [ 0.1362,  0.5293,  0.2273,  ...,  0.0099,  0.1104,  1.6830],
         ...,
         [-0.9282,  1.5452,  0.3153,  ...,  0.0812,  0.0172,  0.0926],
         [ 0.7231,  1.6143,  0.4003,  ..., -0.8685, -0.0963, -1.7352],
         [-1.0041, -0.2028,  1.2649,  ..., -0.0769,  1.6064, -0.4365]],
        device='cuda:0'),
 tensor([-0.4431,  0.0979,  0.8685,  ..., -0.3213, -0.6257,  1.873

In [43]:
subset_sizes = [1000, 2000, 5000]

for subset_size in subset_sizes:
  train_x, test_x, train_y, test_y, _, _ = load_data(filepath="/content/drive/MyDrive/datasets/robot_arm.npz", subset_size=subset_size)
  model, likelihood, training_time = train_exactGP(train_x, train_y, epochs=10)
  metrics = gp_eval(model, likelihood, test_x, test_y)
  print(f"Subset Size: {subset_size}")
  print(f"Training Time: {training_time:.2f}s")
  print(f"RMSE: {metrics[0]:.4f}")
  print(f"NLL: {metrics[1]:.4f}")
  print(f"Calibration: {metrics[2]:.4f}")

Shape: X=(8192, 8), y=(8192,)
Subset: X=(1000, 8), y=(1000,)
Epoch: 10/10, Loss: 1.0269055366516113
Training time - 0.09s
Subset Size: 1000
Training Time: 0.09s
RMSE: 0.5091
NLL: 157.3750
Calibration: 0.9900
Shape: X=(8192, 8), y=(8192,)
Subset: X=(2000, 8), y=(2000,)
Epoch: 10/10, Loss: 0.9014069437980652
Training time - 0.23s
Subset Size: 2000
Training Time: 0.23s
RMSE: 0.4153
NLL: 286.7465
Calibration: 0.9975
Shape: X=(8192, 8), y=(8192,)
Subset: X=(5000, 8), y=(5000,)
Epoch: 10/10, Loss: 0.7753798961639404
Training time - 0.48s
Subset Size: 5000
Training Time: 0.48s
RMSE: 0.3314
NLL: 646.2325
Calibration: 1.0000


In [49]:
print("CONCRETE")
load_data(filepath="/content/drive/MyDrive/datasets/concrete.npz", subset_size=None, test_size=0.2)

CONCRETE
Shape: X=(1030, 8), y=(1030,)


(tensor([[-1.1608,  0.8574,  0.9825,  ..., -0.2575, -0.6475, -0.2757],
         [ 1.3086, -0.6025,  1.2326,  ..., -1.9269, -0.2731, -0.2757],
         [-0.0771, -0.8556,  1.0661,  ...,  1.0179,  0.0666, -0.6893],
         ...,
         [-0.8655, -0.8556,  1.1260,  ...,  1.3408,  0.3311,  0.9156],
         [ 1.7832,  0.5111, -0.8311,  ..., -1.5442,  0.1161, -0.2757],
         [ 0.2851, -0.8556,  0.9356,  ..., -0.6173,  0.1346, -0.2757]],
        device='cuda:0'),
 tensor([[-0.1615,  0.4559, -0.8311,  ..., -0.5283, -1.2616,  5.3002],
         [ 0.7373,  1.3187, -0.8311,  ..., -0.3668, -0.2015, -0.6231],
         [ 0.9913,  1.3187, -0.8311,  ..., -0.3668, -0.2015, -0.2757],
         ...,
         [ 0.0274,  1.4556, -0.8311,  ...,  0.3948,  0.3904, -0.6231],
         [ 1.3086, -0.6025,  1.2326,  ..., -1.9269, -0.2731,  0.1876],
         [ 0.5177, -0.8556, -0.8311,  ..., -0.0705,  0.1099,  0.7502]],
        device='cuda:0'),
 tensor([-4.8512e-01,  1.5542e+00, -7.1535e-01, -1.4592e-01, -1.68

In [47]:
for subset_size in subset_sizes:
  train_x, test_x, train_y, test_y, _, _ = load_data(filepath="/content/drive/MyDrive/datasets/concrete.npz", subset_size=subset_size)
  model, likelihood, training_time = train_exactGP(train_x, train_y, epochs=10)
  metrics = gp_eval(model, likelihood, test_x, test_y)
  print(f"Subset Size: {subset_size}")
  print(f"Training Time: {training_time:.2f}s")
  print(f"RMSE: {metrics[0]:.4f}")
  print(f"NLL: {metrics[1]:.4f}")
  print(f"Calibration: {metrics[2]:.4f}")

Shape: X=(1030, 8), y=(1030,)
Subset: X=(1000, 8), y=(1000,)
Epoch: 10/10, Loss: 0.8289266228675842
Training time - 0.09s
Subset Size: 1000
Training Time: 0.09s
RMSE: 0.3928
NLL: 123.9131
Calibration: 1.0000
Shape: X=(1030, 8), y=(1030,)
Epoch: 10/10, Loss: 0.8227713704109192
Training time - 0.21s
Subset Size: 2000
Training Time: 0.21s
RMSE: 0.3816
NLL: 129.3680
Calibration: 1.0000
Shape: X=(1030, 8), y=(1030,)
Epoch: 10/10, Loss: 0.8217010498046875
Training time - 0.19s
Subset Size: 5000
Training Time: 0.19s
RMSE: 0.3814
NLL: 129.4923
Calibration: 1.0000


In [50]:
print("PROTEIN")
load_data(filepath="/content/drive/MyDrive/datasets/protein.npz", subset_size=None, test_size=0.2)

PROTEIN
Shape: X=(45730, 9), y=(45730,)


(tensor([[-1.1438, -0.6733,  1.3608,  ..., -0.7736, -0.5477,  1.3764],
         [-0.5062, -0.6366, -0.5668,  ..., -0.0980, -0.3343,  0.2518],
         [-0.3346, -0.2579,  0.1212,  ...,  0.1484, -0.7788,  0.1132],
         ...,
         [-0.3282, -0.6805, -1.0441,  ..., -0.3944, -0.8677,  0.3763],
         [-0.8380, -0.2112,  1.8404,  ..., -0.8575, -1.1343,  1.3212],
         [-0.5511, -0.6763, -0.5871,  ..., -0.3655, -0.9921,  0.5723]],
        device='cuda:0'),
 tensor([[-0.7005, -0.3832,  0.7451,  ...,  0.0085, -1.1877,  0.7028],
         [ 1.7822,  2.0305,  0.7629,  ...,  0.7980,  1.1590, -1.8750],
         [ 1.5308,  1.9034,  0.9319,  ...,  0.8780,  1.4079, -1.7829],
         ...,
         [-0.3919, -0.4591, -0.3055,  ..., -0.2295,  0.4657,  0.3935],
         [ 1.7062,  2.0048,  0.8294,  ...,  0.6012,  0.6434, -1.6183],
         [-0.7031, -0.3198,  0.9634,  ...,  0.0495, -0.7610, -0.0187]],
        device='cuda:0'),
 tensor([-0.5594, -0.7696,  1.0588,  ...,  1.0207, -0.1574, -0.976

In [51]:
for subset_size in subset_sizes:
  train_x, test_x, train_y, test_y, _, _ = load_data(filepath="/content/drive/MyDrive/datasets/protein.npz", subset_size=subset_size)
  model, likelihood, training_time = train_exactGP(train_x, train_y, epochs=10)
  metrics = gp_eval(model, likelihood, test_x, test_y)
  print(f"Subset Size: {subset_size}")
  print(f"Training Time: {training_time:.2f}s")
  print(f"RMSE: {metrics[0]:.4f}")
  print(f"NLL: {metrics[1]:.4f}")
  print(f"Calibration: {metrics[2]:.4f}")

Shape: X=(45730, 9), y=(45730,)
Subset: X=(1000, 9), y=(1000,)
Epoch: 10/10, Loss: 1.2664333581924438
Training time - 0.09s
Subset Size: 1000
Training Time: 0.09s
RMSE: 0.8040
NLL: 234.2989
Calibration: 0.9500
Shape: X=(45730, 9), y=(45730,)
Subset: X=(2000, 9), y=(2000,)
Epoch: 10/10, Loss: 1.2102549076080322
Training time - 0.20s
Subset Size: 2000
Training Time: 0.20s
RMSE: 0.7495
NLL: 450.0314
Calibration: 0.9500
Shape: X=(45730, 9), y=(45730,)
Subset: X=(5000, 9), y=(5000,)
Epoch: 10/10, Loss: 1.1579676866531372
Training time - 0.47s
Subset Size: 5000
Training Time: 0.47s
RMSE: 0.7021
NLL: 1064.7396
Calibration: 0.9720


In [53]:
print("SARCOS")
load_data(filepath="/content/drive/MyDrive/datasets/sarcos.npz", subset_size=None, test_size=0.2)

SARCOS
Shape: X=(48933, 21), y=(48933,)


(tensor([[ 0.5099,  1.0572, -0.7961,  ...,  1.4406,  0.9679, -1.6581],
         [-0.8785, -0.3972, -2.3847,  ...,  1.9720,  2.5982, -0.4899],
         [-1.4277, -0.9678,  1.0731,  ..., -0.4240,  0.2015,  0.1482],
         ...,
         [-0.3338, -0.4206, -0.2062,  ..., -1.3949,  1.1409,  1.5486],
         [-1.1535, -0.7938,  0.5435,  ..., -1.9183, -1.5790,  1.3357],
         [-0.5671,  0.0440, -1.7767,  ...,  0.0747,  0.2436, -1.2036]],
        device='cuda:0'),
 tensor([[ 0.5365,  0.7983, -0.3437,  ...,  0.3153, -1.0161, -1.6491],
         [ 0.5811,  1.0840,  1.6059,  ..., -0.6889,  0.2703, -0.7412],
         [ 0.5637,  0.8786, -0.3662,  ...,  0.0714,  0.2861, -0.0488],
         ...,
         [ 0.8491,  1.0984,  0.9888,  ...,  0.3438, -0.5824,  0.7398],
         [ 2.4173,  0.6410,  0.5246,  ..., -0.1668,  0.0556,  0.4851],
         [-0.3071,  0.9642, -0.7939,  ...,  1.4792, -1.5855, -1.4379]],
        device='cuda:0'),
 tensor([ 1.3420,  2.7313, -0.6252,  ..., -0.3906, -0.9300, -0.397

In [54]:
for subset_size in subset_sizes:
  train_x, test_x, train_y, test_y, _, _ = load_data(filepath="/content/drive/MyDrive/datasets/sarcos.npz", subset_size=subset_size)
  model, likelihood, training_time = train_exactGP(train_x, train_y, epochs=10)
  metrics = gp_eval(model, likelihood, test_x, test_y)
  print(f"Subset Size: {subset_size}")
  print(f"Training Time: {training_time:.2f}s")
  print(f"RMSE: {metrics[0]:.4f}")
  print(f"NLL: {metrics[1]:.4f}")
  print(f"Calibration: {metrics[2]:.4f}")

Shape: X=(48933, 21), y=(48933,)
Subset: X=(1000, 21), y=(1000,)
Epoch: 10/10, Loss: 1.187052845954895
Training time - 0.09s
Subset Size: 1000
Training Time: 0.09s
RMSE: 0.7095
NLL: 195.7430
Calibration: 0.9700
Shape: X=(48933, 21), y=(48933,)
Subset: X=(2000, 21), y=(2000,)
Epoch: 10/10, Loss: 1.0731534957885742
Training time - 0.19s
Subset Size: 2000
Training Time: 0.19s
RMSE: 0.6448
NLL: 375.7748
Calibration: 0.9725
Shape: X=(48933, 21), y=(48933,)
Subset: X=(5000, 21), y=(5000,)
Epoch: 10/10, Loss: 0.9254888892173767
Training time - 0.51s
Subset Size: 5000
Training Time: 0.51s
RMSE: 0.4189
NLL: 759.9874
Calibration: 0.9910


In [55]:
print("SYNTHETIC")
load_data(filepath="/content/drive/MyDrive/datasets/synthetic_heteroscedastic.npz", subset_size=None, test_size=0.2)

SYNTHETIC
Shape: X=(5000, 2), y=(5000,)


(tensor([[-0.8327, -0.2844],
         [-0.2644,  0.9771],
         [ 0.3035, -0.4082],
         ...,
         [-1.5917,  0.9212],
         [ 0.8969,  0.5512],
         [-0.7564, -0.4181]], device='cuda:0'),
 tensor([[ 0.5876, -1.0854],
         [-0.0192,  0.0443],
         [ 0.4450,  0.7949],
         ...,
         [ 0.4648,  0.2451],
         [ 0.5965, -0.9930],
         [-0.5957,  0.3378]], device='cuda:0'),
 tensor([-0.4159, -1.2998,  1.4912,  ...,  0.6883,  0.6907, -0.6203],
        device='cuda:0'),
 tensor([ 1.0285e+00,  7.5425e-01,  8.9678e-01,  1.7980e+00, -1.9474e-02,
         -6.1743e-01,  3.1872e-01, -1.4240e+00, -1.5981e+00, -8.3311e-01,
         -2.1695e-01, -1.6458e+00, -1.3591e-02, -1.9082e+00,  4.0394e-02,
          6.1075e-01, -1.5826e+00,  5.3169e-01,  9.5945e-02,  1.6668e+00,
         -1.7635e-01, -4.6177e-01,  1.0586e+00, -1.2007e-01,  5.9591e-01,
          1.0696e+00, -5.2949e-02,  5.9446e-01,  5.9145e-01,  3.5968e-01,
         -8.0533e-01,  3.4676e-01, -1.1119e+00

In [56]:
for subset_size in subset_sizes:
  train_x, test_x, train_y, test_y, _, _ = load_data(filepath="/content/drive/MyDrive/datasets/synthetic_heteroscedastic.npz", subset_size=subset_size)
  model, likelihood, training_time = train_exactGP(train_x, train_y, epochs=10)
  metrics = gp_eval(model, likelihood, test_x, test_y)
  print(f"Subset Size: {subset_size}")
  print(f"Training Time: {training_time:.2f}s")
  print(f"RMSE: {metrics[0]:.4f}")
  print(f"NLL: {metrics[1]:.4f}")
  print(f"Calibration: {metrics[2]:.4f}")

Shape: X=(5000, 2), y=(5000,)
Subset: X=(1000, 2), y=(1000,)
Epoch: 10/10, Loss: 0.512779176235199
Training time - 0.09s
Subset Size: 1000
Training Time: 0.09s
RMSE: 0.1765
NLL: 81.0665
Calibration: 1.0000
Shape: X=(5000, 2), y=(5000,)
Subset: X=(2000, 2), y=(2000,)
Epoch: 10/10, Loss: 0.47150835394859314
Training time - 0.23s
Subset Size: 2000
Training Time: 0.23s
RMSE: 0.1910
NLL: 161.5568
Calibration: 1.0000
Shape: X=(5000, 2), y=(5000,)
Epoch: 10/10, Loss: 0.4511450231075287
Training time - 0.48s
Subset Size: 5000
Training Time: 0.48s
RMSE: 0.1637
NLL: 383.9438
Calibration: 1.0000


In [8]:
class RFF(torch.nn.Module):
  def __init__(self, num_features=1000, lengthscale=1.0, sigma_noise=0.1):
    super().__init__()
    self.num_features = num_features
    self.sigma_noise = sigma_noise
    self.lengthscale = lengthscale
    self.weights = None
    self.bias = None
    self.linear_weights = None

  def fit(self, X, y):
    D = X.shape[1]
    self.weights = torch.randn(D, self.num_features, device=X.device) / self.lengthscale
    self.bias = torch.randn(self.num_features, device=X.device) * 2 * np.pi

    Z = self.transform(X)

    jitter = self.sigma_noise**2 * torch.eye(self.num_features, device=X.device)
    self.linear_weights = torch.linalg.solve(Z.T @ Z + jitter, Z.T @ y)

  def transform(self, X):
    projection = X @ self.weights + self.bias
    Z = np.sqrt(2.0/self.num_features) * torch.cos(projection)
    return Z

  def predict(self, X_test):
    Z_star = self.transform(X_test)
    pred_mean = Z_star @ self.linear_weights

    return pred_mean, None

In [9]:
class Nystrom(torch.nn.Module):
  def __init__(self, num_lamdmarks=500, lengthscale=1.0, sigma_noise=0.1):
    super().__init__()
    self.num_lamdmarks = num_lamdmarks
    self.lengthscale = lengthscale
    self.sigma_noise = sigma_noise
    self.landmarks = None
    self.K_mm_inv = None
    self.alpha = None

  def kernel(self, x1, x2):
    dist = torch.cdist(x1, x2, p=2)
    return torch.exp(-0.5 * (dist/self.lengthscale) ** 2)

  def fit(self, X, y):
    mins = X.min(dim=0)[0]
    maxs = X.max(dim=0)[0]
    self.landmarks = torch.rand(self.num_lamdmarks, X.shape[1], device=X.device) * (maxs - mins) + mins

    K_mm = self.kernel(self.landmarks, self.landmarks)
    K_nm = self.kernel(X, self.landmarks)

    K_mm_stable = K_mm + 1e-6 * torch.eye(self.num_lamdmarks, device=X.device)
    L_mm = torch.linalg.cholesky(K_mm_stable)
    Lambda = K_mm_stable + (1.0/self.sigma_noise**2) * (K_nm.t() @ K_nm)

    self.woodbury_inv = torch.linalg.inv(Lambda)
    self.weights = (1.0/self.sigma_noise**2) * (self.woodbury_inv @ K_nm.t() @ y)

  def predict(self, X_test):
    K_tm = self.kernel(X_test, self.landmarks)
    mean = K_tm @ self.weights
    return mean, None

In [57]:
def benchmark(X_full, y_full, subset_sizes=[1000, 2000, 5000]):
  outputs = []
  lengthscale = 1.0
  noise = 0.2

  for subset_size in subset_sizes:
    X = X_full[:subset_size].to(device)
    y = y_full[:subset_size].to(device)
    X_test = X_full[-100].to(device)
    y_test = y_full[-100].to(device)

    #ExactGP
    torch.cuda.empty_cache()
    t0 = time.time()
    model, likelihood, t_train = train_exactGP(X, y, epochs=10)
    t_eval = time.time() - t0

    metrics = gp_eval(model, likelihood, X_test, y_test)
    outputs.append({"method" : "exact GP", "N" : subset_size, "Time": t_eval, "RMSE": metrics["RMSE"]})

    #RFF
    torch.cuda.empty_cache()
    t0 = time.time()
    rff = RFF(num_features=1000, lengthscale=lengthscale, sigma_noise=noise).to(device)
    rff.fit(X, y)
    t_eval_rff = time.time() - t0

    pred_mean, _ = rff.predict(X_test)
    rmse_rff = torch.sqrt(torch.mean((pred_mean - y_test)**2)).item()
    outputs.append({"method" : "RFF", "N" : subset_size, "Time": t_eval_r, "RMSE": rmse_rff})

    #Nystrom
    torch.cuda.empty_cache()
    t0 = time.time()
    nystrom = Nystrom(num_lamdmarks=500, lengthscale=lengthscale, sigma_noise=noise).to(device)
    nystrom.fit(X, y)
    t_eval_nystrom = time.time() - t0

    pred_mean, _ = nystrom.predict(X_test)
    rmse_nystrom = torch.sqrt(torch.mean((pred_mean - y_test)**2)).item()
    outputs.append({"method" : "Nystrom", "N" : subset_size, "Time": t_eval_nystrom, "RMSE": rmse_nystrom})

  return outputs